In [1]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [2]:
def find_league_code(input_league_name):
    search = f"{input_league_name} Standings FbRef"
    url = 'https://www.google.com/search'

    headers = {'Accept' : '*/*', 'Accept-Language': 'en-US,en;q=0.5','User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82',
}
    parameters = {'q': search}

    content = requests.get(url, headers = headers, params = parameters).text
    soup = BeautifulSoup(content, 'html.parser')

    search = soup.find(id = 'search')
    first_link = search.find('a')

    url_tosplit = (first_link['href'])

    code_league = url_tosplit.split('/')[5]
    name_league = url_tosplit.split('/')[6]
    #name_club, code_club
    return code_league, name_league



In [3]:
def format_number_string(number):
    """
    Format a number into a string of the form "number-(number+1)".

    Parameters:
    - number (int): The input number.

    Returns:
    - formatted_string (str): The formatted string.
    """
    formatted_string = f"{number}-{number+1}"
    return formatted_string

In [4]:
def find_table_year(league_code, league_url, season):

    year_string = format_number_string(season)

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = f'https://fbref.com/en/comps/{league_code}/{year_string}/{year_string}-{league_url}'
    pageTree = requests.get (page, headers = headers)
    pageSoup = BeautifulSoup (pageTree.content, 'html.parser')

    country_array = []
    all_tables = pageSoup.findAll("tbody")
    rows = all_tables[0].find_all('tr')

    for row in rows:
        name = row.find('td',{"data-stat":"team"}).text.strip().encode().decode("utf-8")
        plz = row.find('a')['href']
        data = [name, plz.split('/')[3]]
        country_array.append(data[0])
        #Dict_Clubs[data[0]] = data[1]
    return country_array


In [5]:
football_leagues = ['EPL', 'Ligue 1', 'Serie A', 'La Liga', 'Bundesliga', 'Turkish League', 'Eredivisie', 'Portuguese First Division']

years = list(range(2013, 2024))


In [6]:
def create_league_teams_table(input_league):
    large_df = pd.DataFrame()

    league_code_and_url = find_league_code(input_league)

    for i in range(0, len(years)):
        team_list_season = find_table_year(league_code_and_url[0], league_code_and_url[1], years[i])

        league_teams_df = pd.DataFrame({"League": input_league, 
                                    "Season": years[i], 
                                    "index": [i]})

        league_teams_df['Teams'] = league_teams_df.apply(lambda row: team_list_season if row.name == 0 else None, axis=1)

        large_df = pd.concat([large_df, league_teams_df], ignore_index=True)


    return large_df
    

In [39]:
big_df_final#['League'].value_counts()

,League,Season,index,Teams
0,EPL,2013,0,"[Manchester City, Liverpool, Chelsea, Arsenal,..."
1,EPL,2014,1,"[Chelsea, Manchester City, Arsenal, Manchester..."
2,EPL,2015,2,"[Leicester City, Arsenal, Tottenham, Mancheste..."
3,EPL,2016,3,"[Chelsea, Tottenham, Manchester City, Liverpoo..."
4,EPL,2017,4,"[Manchester City, Manchester Utd, Tottenham, L..."
...,...,...,...,...
83,Portuguese First Division,2019,6,"[Porto, Benfica, Braga, Sporting CP, Rio Ave, ..."
84,Portuguese First Division,2020,7,"[Sporting CP, Porto, Benfica, Braga, Paços, Sa..."
85,Portuguese First Division,2021,8,"[Porto, Sporting CP, Benfica, Braga, Gil Vicen..."
86,Portuguese First Division,2022,9,"[Benfica, Porto, Braga, Sporting CP, Arouca, V..."


In [37]:
#big_df_final = pd.DataFrame()
#for i in range(0, len(football_leagues)):
for i in range(6, 8):
    dataframe_to_add = create_league_teams_table(football_leagues[i])

    big_df_final = pd.concat([big_df_final, dataframe_to_add], ignore_index=True)


In [40]:
big_df_final.to_csv("teams in top division by season, 2013-24.csv",index=False)

## Debug

In [9]:
large_df = pd.DataFrame()
input_league = 'EPL'
league_code_and_url = find_league_code(input_league)
for i in range(0, 1):
#for i in range(0, len(years)):
    #team_list_season = find_table_year(league_code_and_url[0], league_code_and_url[1], years[i])

    year_string = format_number_string(years[i])

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = f'https://fbref.com/en/comps/{league_code_and_url[0]}/{year_string}/{year_string}-{league_code_and_url[1]}'
    pageTree = requests.get (page, headers = headers)
    pageSoup = BeautifulSoup (pageTree.content, 'html.parser')

In [8]:
large_df = pd.DataFrame()
input_league = 'EPL'
league_code_and_url = find_league_code(input_league)

for i in range(0, len(years)):
    #team_list_season = find_table_year(league_code_and_url[0], league_code_and_url[1], years[i])

    year_string = format_number_string(years[i])

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = f'https://fbref.com/en/comps/{league_code_and_url[0]}/{year_string}/{year_string}-{league_code_and_url[1]}'
    pageTree = requests.get (page, headers = headers)
    pageSoup = BeautifulSoup (pageTree.content, 'html.parser')

    country_array = []
    all_tables = pageSoup.findAll("tbody")
    rows = all_tables[0].find_all('tr')

    for row in rows:
        name = row.find('td',{"data-stat":"team"}).text.strip().encode().decode("utf-8")
        plz = row.find('a')['href']
        data = [name, plz.split('/')[3]]
        country_array.append(data[0])
        #Dict_Clubs[data[0]] = data[1]



    league_teams_df = pd.DataFrame({"League": input_league, 
                                "Season": years[i], 
                                "index": [i]})

    league_teams_df['Teams'] = league_teams_df.apply(lambda row: country_array if row.name == 0 else None, axis=1)


    large_df = pd.concat([large_df, league_teams_df], ignore_index=True)

large_df

IndexError: list index out of range